In [1]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
from ikpy.chain import Chain #ik

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1) #tenth of a second delay

#Defining variables
angle = 90
s_time = 1000 #milliseconds
gripper = 6

In [ ]:
def home(): #set arm to home position after task is complete. Useful for ik
    Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time)

In [ ]:
# Defining a kinematic chain with 6 URDFLinks detailing the base and 5 moving links with their respective positions in meters, orientations, and rotation axes. Values are based off home function.
dof_chain = chain.Chain(links=[
    link.URDFLink(
      name="base",
      translation_vector=[0, 0, 0], #distance from previous joint to current joint in 3D space 
      orientation=[0, 0, 0], #rotation angles
      rotation=[0, 0, 1], #capability of rotation, for base use z-axis to define the axis around which any potential rotation could occur if it were not fixed. The base itself does not actually rotate but is the reference point from which all other rotations are calculated.
    ),
    link.URDFLink(
      name="first_link",
      translation_vector=[0, 0, 0.5], # meters from base
      orientation=[0, 0, 0],
      rotation=[0, 1, 0],
    ),
    link.URDFLink(
      name="second_link",
      translation_vector=[0, 0, 0.5], # meters from first link 
      orientation=[0, 0, 0],
      rotation=[0, 1, 0],
    ),
    link.URDFLink(
      name="third_link",
      translation_vector=[0, 0, 0.5], # meters from second link
      orientation=[0, 0, 0],
      rotation=[0, 1, 0],
    ),
    link.URDFLink(
      name="fourth_link",
      translation_vector=[0, 0, 0.5], # meters from third link 
      orientation=[0, 0, 0],
      rotation=[0, 1, 0],
    ),
    link.URDFLink(
      name="fifth_link",
      translation_vector=[0, 0, 0.5], # meters from fourth link
      orientation=[0, 0, 0],
      rotation=[0, 1, 0],
    ),
], active_links_mask=[False]) # This parameter is a list of booleans indicating whether a link is active (i.e., whether it should be considered during the inverse kinematics computation). For example, if you have a fixed base that should not move, you mark it as False. An active link (one that moves) is marked as True.

In [ ]:
def convert_to_servo_commands():
    

In [2]:
def home_to_cupstack():
    Arm.Arm_serial_servo_write6(angle-90, angle, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle+90, s_time) #gripper closing 
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-30, angle-75, angle+25, angle, angle+90, s_time) #lifting cup straight up
    time.sleep(1)   

In [1]:
def cupstack_to_dispenser():
    # Define target position
    target_vector = [0.1, 0.1, 0.1] # Desired position of the gripper relative to the base frame, which is at [0, 0, 0]
    target_frame = np.eye(4) # 4x4 matrix for Rot, Pos and an extra dimension for homogeneous coordinates
    #do we need a line for defining the rotations of target frame? 
    target_frame[:3, 3] = target_vector # Selects the first three elements of the fourth column of the target_frame matrix representing the position in space.
    
    #function within ikpy that takes the setup chain and does the math for ik to reach specified target
    ik = dof_chain.inverse_kinematics(target_frame) # Compute the inverse kinematics

    # Translate IK angles to servo commands
    servo_angles = convert_to_servo_commands(ik)
    Arm.Arm_serial_servo_write6(*servo_angles, s_time) # '*' because servo_angles holds a list of angles when storing convert_to_servo_commands(ik)
    
    time.sleep(5) #five sec delay to allow for whole arm to move  

    #Release cup
    Arm.Arm_serial_servo_write(gripper, angle-90, s_time)
    time.sleep(1)  
    

In [2]:
def main():
    home()
    time.sleep(1) #one sec delay
    home_to_cupstack() #(FK)
    home()
    time.sleep(1) 
    cupstack_to_dispenser() #(IK)
    push_to_dispense() #(FK) pushes button to dispense drink 
    home()
    time.sleep(1) 
    your_drink() #(IK) after a certain amount of time the arm will grab the cup from under the dispenser and the arm will fancily show the user the cup it just poured  
    
try :
    main()
except KeyboardInterrupt:
     print(" Program closed! ")
     pass